參考教學： http://blog.castman.net/%E6%95%99%E5%AD%B8/2016/12/20/python-data-science-tutorial-2.html

# requests

In [1]:
import requests

In [2]:
def get_web_page(url):
    resp = requests.get(
    url = url,
    cookies = {"over18":"1"}    #已滿18歲
    )
    
    if resp.status_code !=200:
        print("Invalid url",resp.url)
    else:
        return resp.text

In [3]:
# get_web_page("https://www.ptt.cc/bbs/Beauty/index.html")

# BeautifulSoup

In [4]:
#你在瀏覽器看到的美觀網頁，主要由三個部分構成: HTML (網頁的骨架結構)、CSS (網頁的樣式) 與 JavaScript (在瀏覽器端執行，負責與使用者互動的程式功能)。
#對於網頁或爬蟲的初學者來說，最重要的觀念是了解：
#網頁就是由各式標籤 (tag) 所組成的階層式文件，要取得所需的網頁區塊資料，只要用 tag 與相關屬性去定位資料所在位置即可。

In [5]:
#HTML 文件內不同的標籤 (例如 <title>, <h1>, <p>, <a> 有著不同的語義，表示建構網頁用的不同元件，
#且標籤可以有各種屬性 (例如 id, class, style 等通用屬性, 或 href 等專屬屬性)，

In [6]:
#因此我們可以用標籤 + 屬性去定位資料所在的區塊並取得資料

In [7]:
from bs4 import BeautifulSoup

In [8]:
html_doc = """
<html>
  <head>
    <title>我是網頁標題</title>
    <style>
    .large {
      color:blue;
      text-align: center;
    }
    </style>
  </head>
  <body>
    <h1 class="large">我是變色且置中的抬頭</h1>
    <p id="p1">我是段落一</p>
    <p id="p2" style="">我是段落二</p>
    <div><a href='http://blog.castman.net' style="font-size:200%;">我是放大的超連結</a></div>
  </body>
</html>
"""

In [9]:
soup = BeautifulSoup(html_doc, 'html.parser')  #把它變為美麗的一碗濃湯
print(soup)


<html>
<head>
<title>我是網頁標題</title>
<style>
    .large {
      color:blue;
      text-align: center;
    }
    </style>
</head>
<body>
<h1 class="large">我是變色且置中的抬頭</h1>
<p id="p1">我是段落一</p>
<p id="p2" style="">我是段落二</p>
<div><a href="http://blog.castman.net" style="font-size:200%;">我是放大的超連結</a></div>
</body>
</html>



In [10]:
# <title>, <h1>, <p>, <a>  +  id, class, style 等通用屬性, 或 href 

soup.find_all("p",id="p1") # e.g.

[<p id="p1">我是段落一</p>]

In [11]:
haha =  soup.find_all("p") #回傳一個list

for p in haha:
    print(p['id'], p.string)

p1 我是段落一
p2 我是段落二


In [12]:
#如果你要取得的屬性不存在，直接使用屬性名稱會出現錯誤訊息，因此若你不確定屬性是否存在，可以改用 get() 方法
print(soup.find(id = "p1").get('style'))

None


### ppt表特版  ( 抓取今天所有的貼文 標題, 推文數量, 網址

In [54]:
#檢視網頁原始碼, 看結構

In [13]:
def get_articles(dom,date):
    soup = BeautifulSoup(dom,"html.parser")
    articles = [] #容器
    divs = soup.find_all("div", "r-ent")  # list
    
    for d in divs:
        if d.find("div","date").string.strip()==date: # 確認日期正確, strip()是去除左右空白格
  
            #取得推文數
            push_count =0
            if d.find("div","nrec").string:  #確認存在
                try:
                    push_count = int(d.find("div","nrec").string)
                except:
                    pass
            
            #取得文章連結&標題
            if d.find("a"): #確認有超連結，表示文章未被刪除
                href = d.find("a")["href"]
                title = d.find("a").string
                articles.append({"title":title,
                                "href":href,
                                "push_count":push_count})
            
    return articles
        
    
    
    

In [14]:
page = get_web_page('https://www.ptt.cc/bbs/Beauty/index2201.html')

In [15]:
import time
if page:
    date = time.strftime("%m/%d").lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式。lstrip的l是左邊的意思
    current_articles = get_articles(page, date)
    for post in current_articles:
        print(post)

In [16]:
# 上面那行已經過期ＱＱＱ

import time
if page:
    date = "6/26"  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式。lstrip的l是左邊的意思
    current_articles = get_articles(page, date)
    for post in current_articles:
        print(post)

{'push_count': 10, 'title': '[正妹] 一張試水溫', 'href': '/bbs/Beauty/M.1498408545.A.DDF.html'}
{'push_count': 10, 'title': '[正妹]冰與火 Ros', 'href': '/bbs/Beauty/M.1498411524.A.D20.html'}
{'push_count': 0, 'title': '[神人] 應該是常外拍的model?', 'href': '/bbs/Beauty/M.1498414669.A.9AD.html'}
{'push_count': 0, 'title': '[正妹] 朋友來試水溫', 'href': '/bbs/Beauty/M.1498420374.A.620.html'}
{'push_count': 12, 'title': '[正妹] 藍色星期一', 'href': '/bbs/Beauty/M.1498442575.A.20B.html'}
{'push_count': 13, 'title': '[正妹] 三鐵健康女孩', 'href': '/bbs/Beauty/M.1498446627.A.223.html'}
{'push_count': 0, 'title': '[神人] 舞台車正妹', 'href': '/bbs/Beauty/M.1498446757.A.85D.html'}
{'push_count': 0, 'title': '[正妹] 衝突美 x 韓國 x 健身', 'href': '/bbs/Beauty/M.1498450082.A.644.html'}
{'push_count': 14, 'title': '[正妹] Nathana Schilling 巴西模特兒 178cm ', 'href': '/bbs/Beauty/M.1498451998.A.795.html'}
{'push_count': 38, 'title': '[正妹] 老師好甜', 'href': '/bbs/Beauty/M.1498455665.A.833.html'}


### 抓取今天貼文的圖，一個貼文有一個資料夾 裡面放很多圖  and  計算圖片數量

In [17]:
import re

In [18]:
import os

In [19]:
import urllib

In [20]:
# 回傳該篇文章的圖片連結
def parse(dom):
    soup = BeautifulSoup(dom, 'html.parser')
    links = soup.find(id='main-content').find_all('a')
    img_urls = []
    for link in links:
        if re.match(r'^https?://(i.)?(m.)?imgur.com', link['href']):  # 此串網址用regular expression海納百川。r是為了逃脫字元
            img_urls.append(link['href'])
    return img_urls

In [21]:
# 儲存圖片
def save(img_urls, title):
    if img_urls:
        try:
            dname = title.strip()  # 用 strip() 去除字串前後的空白
            os.makedirs(dname)   # 在現在路徑底下創建資料夾
            for img_url in img_urls:
                if img_url.split('//')[1].startswith('m.'):
                    img_url = img_url.replace('//m.', '//i.')
                if not img_url.split('//')[1].startswith('i.'):  # if not好用！
                    img_url = img_url.split('//')[0] + '//i.' + img_url.split('//')[1]
                if not img_url.endswith('.jpg'):
                    img_url += '.jpg'
                fname = img_url.split('/')[-1]
                urllib.request.urlretrieve(img_url, os.path.join(dname, fname)) #這行好神秘
        except Exception as e:
            print(e)
            
#下載圖片時用的網址必須是 i.imgur.com 開頭，因此要把 m.imgur.com 換成 i.imgur.com，或把 imgur.com 補成 i.imgur.com

In [23]:
PTT_URL = 'https://www.ptt.cc' #PTT 網頁內文章的 href 屬性是相對路徑，因此連線時要加上完整網址名稱 (PTT_URL)

for article in current_articles:
    page = get_web_page(PTT_URL + article['href'])
    if page:
        #第1步驟
        img_urls = parse(page)
        #2步驟
        save(img_urls, article['title'])
        article['num_image'] = len(img_urls)    # 直接在dictionary創建num_image

In [255]:
# 把資訊存成json檔案

import json

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(current_articles, f, indent=2, sort_keys=True, ensure_ascii=False)
    


In [24]:
#這就是Json檔長的樣子，{}裡面是物件 有點像字典   []是陣列，裏頭可以是幾乎任何東西

current_articles

[{'href': '/bbs/Beauty/M.1498408545.A.DDF.html',
  'num_image': 1,
  'push_count': 10,
  'title': '[正妹] 一張試水溫'},
 {'href': '/bbs/Beauty/M.1498411524.A.D20.html',
  'num_image': 16,
  'push_count': 10,
  'title': '[正妹]冰與火 Ros'},
 {'href': '/bbs/Beauty/M.1498414669.A.9AD.html',
  'num_image': 0,
  'push_count': 0,
  'title': '[神人] 應該是常外拍的model?'},
 {'href': '/bbs/Beauty/M.1498420374.A.620.html',
  'num_image': 8,
  'push_count': 0,
  'title': '[正妹] 朋友來試水溫'},
 {'href': '/bbs/Beauty/M.1498442575.A.20B.html',
  'num_image': 1,
  'push_count': 12,
  'title': '[正妹] 藍色星期一'},
 {'href': '/bbs/Beauty/M.1498446627.A.223.html',
  'num_image': 8,
  'push_count': 13,
  'title': '[正妹] 三鐵健康女孩'},
 {'href': '/bbs/Beauty/M.1498446757.A.85D.html',
  'num_image': 0,
  'push_count': 0,
  'title': '[神人] 舞台車正妹'},
 {'href': '/bbs/Beauty/M.1498450082.A.644.html',
  'num_image': 5,
  'push_count': 0,
  'title': '[正妹] 衝突美 x 韓國 x 健身'},
 {'href': '/bbs/Beauty/M.1498451998.A.795.html',
  'num_image': 7,
  'push_count

# 資料分析

In [ ]:


# http://blog.castman.net/%E6%95%99%E5%AD%B8/2016/12/31/python-data-science-tutorial-5.html